#Бенчмарк. Определение остановок автобусов.

Ночь Анализа Данных Яндекса, 2015

Подключим необходимые библиотеки:

In [2]:
%pylab inline

import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier

Populating the interactive namespace from numpy and matplotlib


Считаем файлы с обучающей и тестовой выборкой, а также файл с остановками.

In [3]:
dtrain = pd.read_csv('gps_train.tsv', sep = '\t', names = ['datetime', 'id', 'type', 'hash', 'latitude', 'longitude'])
dtest = pd.read_csv('gps_test.tsv', sep = '\t', names = ['datetime', 'id', 'type', 'hash', 'latitude', 'longitude'])

dstops = pd.read_csv('stops_train.tsv', sep = '\t', names = ['latitude', 'longitude'])

Создадим массивы с координатами точек.

In [4]:
coordtrain = np.array(dtrain.ix[:, 4:])
coordtest = np.array(dtest.ix[:, 4:])

coordstops = np.array(dstops.ix[:, :])

Создадим обучающую выборку.
Для этого пометим как остановкии те точки маршрутов, которые наиболее близки к реальным остановкам.

In [5]:
nbrs = NearestNeighbors(n_neighbors = 1, algorithm = 'ball_tree').fit(coordtrain)
selected = nbrs.kneighbors(coordstops)[1][:, 0]

ytrain = np.zeros(len(dtrain))
ytrain[selected] = 1

В качестве признаков возьмем расстояния до 5 ближайших точек из обучающей и тестовой выборки соответственно.

In [6]:
nbrs = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree').fit(coordtrain)
Xtrain = nbrs.kneighbors(coordtrain)[0]

nbrs = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree').fit(coordtest)
Xtest = nbrs.kneighbors(coordtest)[0]

Обучим модель случайного леса на наших данных.

In [7]:
model = RandomForestClassifier(n_estimators = 10, random_state = 0, n_jobs = -1)
model.fit(Xtrain, ytrain)

result = model.predict_proba(Xtest)[:, 1]

В качестве результата возьмем 3000 наиболее вероятных точек.
Выведем результат в файл.

In [8]:
np.savetxt('output.txt', coordtest[argsort(result)[-3000:]], header = '3000', comments = '')

In [6]:
!head output.txt

3000
6.266293000000000291e+01 5.866182199999999369e+01
6.303820400000000035e+01 5.836750200000000177e+01
6.306890100000000388e+01 5.835960099999999784e+01
6.292370999999999981e+01 5.855505000000000138e+01
6.309207099999999713e+01 5.796597900000000436e+01
6.294864300000000412e+01 5.853920799999999502e+01
6.303303299999999609e+01 5.825123100000000420e+01
6.275014499999999629e+01 5.863042300000000040e+01
6.292258900000000210e+01 5.850721200000000266e+01


In [1]:
oauth=...

In [2]:
from contest_yandex import YaContestSubmitter
contest = YaContestSubmitter(oauth_token=oauth, contest_id=1304)

In [3]:
r = contest.submit('output.txt')
print r

769435


In [5]:
score, message = contest.get_result()
print "Score: {}, message: {}".format(score, message)

Score: 0.0, message: points 0.0 [*] Correct stops 0 of 1143 in public set.

